<img src='op2-u02.png'/>
<h2><font color='#7F0000'>OP2-08-Banco de Dados</font></h2>

## SQLite

<p>Segundo sua documentação: <i>"SQLite é uma biblioteca em linguagem C que implementa um
    mecanismo de banco de dados SQL pequeno, rápido, independente, de alta confiabilidade e 
    recursos completos"</i>
    (Fonte: <a href='https://sqlite.org/index.html'>https://sqlite.org/index.html</a>).</p>
<p>É, possivelmente, o SGBDR mais usado no mundo, pois está integrado a plataforma Android 
    e pode ser usado em praticamente qualquer sistema operacional, incluindo dispositivos 
    móveis e computadores.</p>
<p>O Python dispõe de um módulo que provê integração simples com o SQLite.</p>

In [ ]:
# importação do módulo SQLite3
import sqlite3

### Conexão com banco de dados

In [ ]:
# Conexão com banco de dados SQLite
# Se o banco de dados não existir, ele será criado
con = sqlite3.connect('db/metereologia.db')
con

In [ ]:
# A partir da conexão, usualmente cria-se um cursor para
# possibilitar a execução de comandos SQL no BD.
cursor = con.cursor()
cursor

### Execução de comandos SQL

In [ ]:
# É conveniente definir string para os comandos que serão usados.
# Remova a tabela estações quando quiser refazer o notebook
# (ou modificar a estrutura da tabela).

# SQL DDL (remoção de tabela) 
sql0 = ['DROP TABLE ESTACOES',
        'DROP TABLE PARAMETROS',
        'DROP TABLE DADOS']

# Execução de comando SQL requer um cursor.
for cmd in sql0:
    try:
        cursor.execute(cmd)
    except Exception as exc:
        print(exc)
# SQL DTL (consolidação de operações)
con.commit()

In [ ]:
# SQL DDL (criação de tabela)
sql1 = '''CREATE TABLE ESTACOES (
    Codigo varchar(8) PRIMARY KEY,
    Nome varchar(40) not null,
    Latitude float not null,
    Longitude float not null,
    Altitude integer not null
)'''
print(sql1)

In [ ]:
# Execução do comando SQL é feita com cursor
cursor.execute(sql1)

In [ ]:
# SQL DDL (criação de tabela)
sql2 = '''CREATE TABLE PARAMETROS (
    Coluna varchar(15) PRIMARY KEY,
    Descricao varchar(40) not null,
    Unidade varchar(5) not null
)'''
print(sql2)

In [ ]:
# SQL DDL (criação de tabela)
sql3 = '''CREATE TABLE DADOS (
    Codigo varchar(8),
    Data date not null,
    Precip_Tot float null,
    Temp_Max float null,
    Temp_Med float null,
    Temp_Min float null,
    Umid_Relat float null,
    PRIMARY KEY (Codigo, Data), 
    FOREIGN KEY (Codigo) references Estacoes(Codigo)
)'''
print(sql3)

In [ ]:
# Execução do comando SQL é feita om cursor
cursor.execute(sql2)
cursor.execute(sql3)

In [ ]:
# Consolidação das tabelas
con.commit()

### Consultas

In [ ]:
# SQL DQL (consulta a dados)
sql4 = 'SELECT Codigo, Nome, Latitude, Longitude, Altitude FROM ESTACOES'

In [ ]:
# Realização de consulta também emprega cursor
cursor.execute(sql4)
# Dados da consulta realizada devem ser recuperados
dados = cursor.fetchall()
type(dados)

In [ ]:
# Define função para exibir dados da tabela Estacoes
def lista_estacoes(dados):
    # Exibição dos dados consultados de maneira tabular
    print('| %-8s | %-40s | %-12s | %-12s | %-8s |' % ('Código', 'Nome', 'Latitude', 'Longitude', 'Altitude'))
    print('+-%8s-+-%40s-+-%12s-+-%12s-+-%8s-+' % (8*'-', 40*'-', 12*'-', 12*'-', 8*'-'))
    for registro in dados:
        print('| %-8s | %-40s | %12.6f | %12.6f | %8d |' % registro)
    print('+-%8s-+-%40s-+-%12s-+-%12s-+-%5s-+' % (8*'-', 40*'-', 12*'-', 12*'-', 8*'-'))
    return

In [ ]:
# Mostra dados da tabela estações
lista_estacoes(dados)

### Inclusão de dados

In [ ]:
# Inserçao de dados em tabela
# SQL DDL (inserção/criação de dados)
sql5 = "INSERT INTO ESTACOES(Codigo, Nome, Latitude, Longitude, Altitude) "\
        "VALUES('A744', 'BRAGANCA PAULISTA', -22.951944, -46.530556, 891)"
# Observe o uso de aspas duplas (para definir a string Python)
# e aspas simples (para delimitar string no SQL)

In [ ]:
# Realização de inserção, como sempre, emprega cursor
cursor.execute(sql5)
# Consolida inserção
con.commit()

In [ ]:
# Realização de consulta também emprega cursor
cursor.execute(sql4)
# Dados da consulta realizada devem ser recuperados
dados = cursor.fetchall()
type(dados)

In [ ]:
# Mostra dados da tabela estações
lista_estacoes(dados)

In [ ]:
sql6 = "INSERT INTO ESTACOES(Codigo, Nome, Latitude, Longitude, Altitude) "\
        "VALUES('A706', 'CAMPOS DO JORDAO', -22.7502777, -45.6038888, 891)"
sql7 = "INSERT INTO ESTACOES(Codigo, Nome, Latitude, Longitude, Altitude) "\
        "VALUES('A701', 'SAO PAULO - MIRANTE', -23.496294, 46.620088, 786)"

In [ ]:
# Realização de inserção, como sempre, emprega cursor
cursor.execute(sql6)
cursor.execute(sql7)
# Consolida inserção
con.commit()

In [ ]:
# Realização de consulta também emprega cursor
cursor.execute(sql4)
# Dados da consulta realizada devem ser recuperados
dados = cursor.fetchall()
# Mostra dados da tabela estações
lista_estacoes(dados)

### Desconexão

In [ ]:
# Fechar a conexão encerra os cursores abertos.
# Não esqueça de consolidar as ações realizadas.
con.close()

### Inserção de múltiplos dados

In [ ]:
# Obtenção de conexão
con = sqlite3.connect('db/metereologia.db')
# Criação do cursor
cursor = con.cursor()

In [ ]:
# importação de módulo para leitura de CSV
from csv import reader

In [ ]:
# Definição de comando SQL parametrizável
sql8 = 'INSERT INTO Dados (Codigo,Data,Precip_Tot,Temp_Max,Temp_Med,Temp_Min,Umid_Relat) ' \
       'values (?, ?, ?, ?, ?, ?, ?)'

In [ ]:
# Funcão que efetua leitura do arquivo CSV inserindo
def inserir_dados(nome_arquivo, estacao):
    with open(nome_arquivo, 'r') as file_reader:
        # Usa file handler para criação de um leitor de CSV
        csv_reader = reader(file_reader, delimiter=';')
        # Usa objeto reader para criar lista de listas com list()
        list_of_rows = list(csv_reader)
        list_of_rows.pop(0) # remove o cabeçalho
        # Percorre lista de listas, efetuando uma inserçao
        for row in list_of_rows:
            print(row)
            # Executa instrução SQL inserindo parâmetros
            cursor.execute(sql8, row)
    # Consolida transação
    cursor.connection.commit()
    return

In [ ]:
inserir_dados('arquivos/dados_A701_D_2021-01-01_2021-03-06.csv', 'A701')
inserir_dados('arquivos/dados_A744_D_2021-01-01_2021-03-06.csv', 'A744')
# https://bdmep.inmet.gov.br/#

In [ ]:
# SQL DQL (consulta a dados)
sql9 = 'SELECT * FROM DADOS'
# Execução do comando SQL
cursor.execute(sql9)
# Recuperação dos dados
dados = cursor.fetchall()

In [ ]:
dados

In [ ]:
# Define função para exibir dados da tabela Dados
def lista_dados(dados):
    # Exibição dos dados consultados de maneira tabular
    print('| %-8s | %-10s | %-10s | %-10s | %-10s | %-10s | %-10s |' %
          ('Estacao', 'Data', 'Precip', 'TMax', 'TMed', 'TMin', 'Umid'))
    print('+-%8s-+-%10s-+-%10s-+-%10s-+-%10s-+-%10s-+-%10s-+' % 
          (8*'-', 10*'-', 10*'-', 10*'-', 10*'-', 10*'-', 10*'-'))
    for registro in dados:
        print('| %-8s | %-10s | %10s | %10s | %10s | %10s | %10s |' % registro)
    print('+-%8s-+-%10s-+-%10s-+-%10s-+-%10s-+-%10s-+-%10s-+' % 
          (8*'-', 10*'-', 10*'-', 10*'-', 10*'-', 10*'-', 10*'-'))
    return

In [ ]:
lista_dados(dados[:10]+dados[-10:])

In [ ]:
sql10 = "SELECT * FROM DADOS WHERE Precip_Tot > 40"
cursor.execute(sql10)
dados_parciais = cursor.fetchall()
lista_dados(dados_parciais)

In [ ]:
# Fechar a conexão encerra os cursores abertos.
# Não esqueça de consolidar as ações realizadas.
con.close()

### FIM
### <a href="http://github.com/pjandl/opy2">Oficina Python Intermediário</a>